In [6]:
import geopandas as gpd
import time
import os
import json
import requests
from osgeo import gdal
import rasterio
import rasterio.mask

import pandas as pd
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import os
import requests
PLANET_API_KEY = "17de9ecc82734c1caf0ef0da1bf90d97"
import datetime


In [7]:
# Set filepath
fp = "shapes/trial-design-n-2021.shp"
# Read file using gpd.read_file()
trial_design = gpd.read_file(fp)
#str(data.iloc[0]['geometry'])
# check the data
#gpd.GeoSeries([data.iloc[0]['geometry']])
trial_design.head(2)

,geometry
0,"POLYGON ((-89.06084 39.72437, -89.06084 39.724..."
1,"POLYGON ((-89.06172 39.72437, -89.06172 39.724..."


In [8]:
df = pd.read_csv('data/res.csv')
print(len(df.index))
df.head(3)
ids = [v.id for k,v in df.iterrows()]

110


In [16]:
for id_ in ids:
    with rasterio.open(v['filename']) as src:
        outpath = f"data/cropped/{id_}"
        if not os.path.exists(outpath):
            # Create a new directory because it does not exist 
            os.makedirs(outpath)
        i = 0
        for geom in trial_design.iterrows():
            out_image, out_transform = rasterio.mask.mask(src, geom[1], crop=True)
            out_meta = src.meta
            out_meta.update(
                {
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                }
            )
            filename = f"{outpath}/{i}.tiff"
            with rasterio.open(filename, "w", **out_meta) as dest:
                dest.write(out_image)
            i += 1
                        
            # Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
            with rasterio.open(filename) as src2:
                band_red = src.read(3)

            with rasterio.open(filename) as src2:
                band_nir = src.read(4)


            xmldoc = minidom.parse("20161218_101700_0e0d_3B_AnalyticMS_metadata.xml")
            nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

            # XML parser refers to bands by numbers 1-4
            coeffs = {}
            for node in nodes:
                bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
                if bn in ['1', '2', '3', '4']:
                    i = int(bn)
                    value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
                    coeffs[i] = float(value)

